<a href="https://colab.research.google.com/github/CaeltUNIFEI/Infografico/blob/main/tratamentoDadosUNIFEI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inicialização

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, date, timedelta
pd.set_option('display.max_columns', 500)

quantidadeLinhasExcluidas = 0

In [ ]:
dados = pd.read_csv('/content/drive/MyDrive/censoUNIFEI/Censo UNIFEI.csv', sep=',', encoding='utf-8')

# Extração e limpeza dos dados

In [ ]:
dados1 = dados.drop(columns=['Carimbo de data/hora', 'Termo de Consentimento e Livre Esclarecimento:'], )
dados1 = dados1.rename(columns={'Data de nascimento:' : 'dataNascimento', 'Sexo:' : 'sexo', 'Orientação sexual:' : 'orientacaoSexual', 'Etinia ou cor:' : 'etnia/cor', 'Religião:' : 'religiao', 'Município de nascimento:' : 'municipioNascimento', 'Unidade Federativa: ' : 'UF', 'Curso:' : 'curso', 'Data de ingresso:' : 'dataIngresso', 'Cursou o ensino médio em:' : 'EM', 'Na forma de ingresso, você utilizou qual categoria?' : 'categoriaIngresso', 'Caso tenha utilizado a cota, qual grupo você selecionou?' : 'grupoCota', 'Já trancou o curso?' : 'trancouCurso', 'Mora em República?' : 'republica', 'Você possui acesso a serviços básicos como água encanada, energia elétrica e coleta de lixo?' : 'servicosBasicos', 'Você trabalha?' : 'trabalha', 'se você trabalha, você tem acesso a benefícios como plano de saúde ou vale-refeição no seu trabalho?' : 'beneficiosTrabalho', 'Qual é a renda total mensal de sua família, considerando todas as fontes de renda, incluindo salários, aposentadorias, pensões, benefícios sociais, entre outros?' : 'rendaMensal', 'Quantas pessoas contribuem para a renda da sua família?' : 'contribuintes', 'Você ou sua família possuem dívidas ou empréstimos? ' : 'dividas/emprestimos', 'Recebe auxílio da UNIFEI?' : 'recebeAuxilio', 'Se recebe auxílio da UNIFEI, qual é?' : 'qualAuxilio', 'O que acha do preço do Restaurante Universitário da UNIFEI?' : 'precoRU', 'O que acha do suporte socioeconômico da UNIFEI?' : 'suporteSocioeconomico', 'Justifique a resposta anterior' : 'justifique', 'Utiliza de assistência da DAE?' : 'assistenciaDAE', 'Possui alguma necessidade física ou psicológica?' : 'possuiNecessidade', 'Se possui alguma necessidade física ou psicológica, qual(is) é(são)?' : 'qualNecessidade'})


In [ ]:
dados1['sexo'] = dados1['sexo'].replace({'Femenino' : 'Feminino'})

# Transformação dos dados

Idade (em anos)

In [ ]:
idadeArray = []

for data in dados1['dataNascimento']:
  idade = int(((date.today() - datetime.strptime(data, '%Y-%m-%d').date()).days)/365)
  idadeArray.append(idade)

dados1.insert(loc= 1, column= 'idade', value=idadeArray)


Data de ingresso

In [ ]:
tempoArrayAnos = []

for data in dados1['dataIngresso']:
  tempo = (date.today() - datetime.strptime(data, '%Y-%m-%d').date())
  tempoAnos = round(tempo.days / 365, 2)

  tempoAnos = np.around(tempoAnos / 0.5) * 0.5
  tempoArrayAnos.append(tempoAnos)

dados1.insert(loc= 10, column= 'tempoNaUnifeiEmAnos', value=tempoArrayAnos)

In [ ]:
print(f"Linha(s) removida(s): {dados1.query('idade < 18').shape[0]}")
dados1.drop(dados1[dados1['idade'] < 18].index, inplace=True) #Remove a linha onde a idade é menor que 18 anos

#print(f"Linha(s) removida(s): {dados1.query('tempoNaUnifeiEmAnos > 15').shape[0]}")
#dados1.drop(dados1[dados1['tempoNaUnifeiEmAnos'] > 15].index, inplace=True)

Linha(s) removida(s): 1


# Contagem dos dados

In [ ]:
contagemIdade = dados1['idade'].value_counts().sort_index()
contagemSexo = dados1['sexo'].value_counts()
contagemOrientacao = dados1['orientacaoSexual'].value_counts()
contagemEtnia = dados1['etnia/cor'].value_counts()
contagemUF = dados1['UF'].value_counts()
contagemTempoUnifei = dados1['tempoNaUnifeiEmAnos'].value_counts().sort_index()

In [ ]:
respostas = dados1['religiao']

mapeamento = {
    "ateísta": "Ateu", "ateu": "Ateu",
    "agnóstico": "Agnóstico",
    "católico": "Católico", "catolica": "Católico", "catolicismo": "Católico", "católica": "Católico", "catolico" : "Católico",
    "evangélico": "Evangélico", "evangélica": "Evangélico", "cristão protestante": "Evangélico", "cristã/evangélica": "Evangélico",
    "espiritismo": "Espiritismo",
    "cristão" : "Cristão",
    "nenhuma": "Sem religião", "não aplica": "Sem religião", "não possuo": "Sem religião", "não possuo religião.": "Sem religião", "nao tenho": "Sem religião", "desvinculado": "Sem religião", "não tenho" : "Sem religião", "não possuo religião" : "Sem religião", "sem religião" : "Sem religião", "não sei" : "Sem religião",
}

def normalizar_respostas(resposta):
    resposta = resposta.strip().lower()
    return mapeamento.get(resposta, '[ERRO]')

dados1['religiao'] = dados1['religiao'].apply(normalizar_respostas)
contagemReligiao = dados1['religiao'].value_counts()

In [ ]:
print("Linha(s) removida(s): " + str(dados1.query('religiao == "Cristão"').shape[0]))
dados1.drop(dados1[dados1['religiao'] == 'Cristão'].index, inplace=True)

Linha(s) removida(s): 2


In [ ]:
dados1.to_csv('/content/drive/MyDrive/censoUNIFEI/dadosLimpos.csv', sep=',', index=False, encoding='utf-8', decimal='.')

quantidadeLinhasExcluidas = dados.shape[0] - dados1.shape[0]

#Nao precisa adicionar +1 por causa do indice 0, ele retorna corretamente.
print(f"Quantidades de \"Entrevistados\": {dados.shape[0]}")
print(f"Quantidades de linhas excluidas: {quantidadeLinhasExcluidas}")
print(dados1.shape[0])

dados1

Quantidades de "Entrevistados": 46
Quantidades de linhas excluidas: 3
43


,dataNascimento,idade,sexo,orientacaoSexual,etnia/cor,religiao,municipioNascimento,UF,curso,dataIngresso,tempoNaUnifeiEmAnos,EM,categoriaIngresso,grupoCota,trancouCurso,republica,servicosBasicos,trabalha,beneficiosTrabalho,rendaMensal,contribuintes,dividas/emprestimos,recebeAuxilio,qualAuxilio,precoRU,suporteSocioeconomico,justifique,assistenciaDAE,possuiNecessidade,qualNecessidade
0,1993-01-02,32,Masculino,Heterossexual,Branca,Ateu,Campinas,São Paulo (SP),Engenharia Eletrônica,2020-01-01,5.0,Escola Privada,Ampla Concorrência,NaN,Sim,Não,Sim,Não,NaN,Mais do que cinco salários mínimos,1,Sim,Não,NaN,Muito caro,Muito ruim,Valores baixos e falta de moradia a ser oferec...,Não,Sim,"Depressão, ansiedade, deficit de atenção"
1,2001-07-24,23,Masculino,Heterossexual,Branca,Sem religião,Itajubá,Minas Gerais (MG),ELT,2024-01-01,1.0,Escola Pública,Ampla Concorrência,NaN,Não,Não,Sim,"Sim, sem receber salário",Não,Entre dois e três salários mínimos,2,Não,Não,NaN,Muito caro,Bom,Considero bom mesmo sem usufruir do suporte já...,Não,Não,NaN
2,2000-11-04,24,Masculino,Heterossexual,Branca,Agnóstico,Itajubá,Minas Gerais (MG),Engenharia Eletrônica,2021-03-01,4.0,Escola Pública,Cota,GRUPO L1 – Candidatos com renda familiar bruta...,Não,Não,Sim,Não,NaN,Entre três e quatro salários mínimos,2,Sim,Sim,Auxílio permanência e alimentação,Muito caro,Razoável,"Acredito que eles tem bons funcionários, porém...",Sim,Não,NaN
3,2000-05-09,24,Masculino,Homossexual,Branca,Sem religião,Paraisópolis,Minas Gerais (MG),Engenharia Eletrônica,2021-03-01,4.0,Escola Privada,Ampla Concorrência,NaN,Sim,Não,Sim,Não,Não,Entre meio e um salário mínimo,1,Sim,Sim,Permanência e Alimentacao,Caro,Bom,Graças ao auxílio eu consigo me manter em Itaj...,Sim,Não,NaN
4,2003-03-02,21,Masculino,Heterossexual,Parda,Católico,Itajubá,Minas Gerais (MG),Engenharia Eletrônica,2024-03-11,1.0,Escola Pública,Cota,GRUPO L1 – Candidatos com renda familiar bruta...,Não,Não,Sim,Não,NaN,Entre meio e um salário mínimo,1,Não,Não,NaN,Caro,Muito Bom,Ajuda muitos alunos,Não,Não,NaN
5,2003-09-10,21,Masculino,Heterossexual,Branca,Católico,Itajubá,Minas Gerais (MG),Engenharia Eletrônica,2023-03-27,2.0,Escola Privada,Ampla Concorrência,NaN,Não,Não,Sim,Não,NaN,Entre três e quatro salários mínimos,2,Não,Não,NaN,Muito caro,Razoável,A UNIFEI deveria fornecer auxílio estudantil a...,Não,Não,NaN
6,2006-04-07,18,Feminino,Heterossexual,Branca,Católico,Varginha,Minas Gerais (MG),Engenharia Eletrônica,2024-03-11,1.0,Escola Privada,Ampla Concorrência,NaN,Não,Não,Sim,Não,NaN,Entre três e quatro salários mínimos,2,Sim,Não,NaN,Caro,Bom,Não tenho conhecimento sobre,Não,Não,NaN
7,2000-08-01,24,Feminino,Heterossexual,Branca,Católico,Brasopolis,Minas Gerais (MG),Engenharia Eletrônica,2020-03-20,5.0,Escola Pública,Cota,GRUPO L1 – Candidatos com renda familiar bruta...,Não,Não,Sim,"Sim, sem receber salário",Não,Entre dois e três salários mínimos,2,Não,Sim,Alimentação e Moradia,Caro,Bom,"Ajuda os alunos, mas o auxilio moradia por exe...",Sim,Não,NaN
8,2003-06-23,21,Masculino,Heterossexual,Branca,Católico,Maria da Fé - MG,Minas Gerais (MG),Engenharia Eletrônica,2024-03-14,1.0,Escola Pública,Cota,"GRUPO L5 – Candidatos que, independente da ren...",Não,Não,Sim,Não,NaN,Entre dois e três salários mínimos,3,Não,Sim,Alimentação,Muito caro,Bom,Pessoalmente considero o suficiente o que rece...,Não,Não,NaN
9,2003-01-07,22,Masculino,Heterossexual,Branca,Católico,Itanhandu,Minas Gerais (MG),Engenharia Eletrônica,2022-08-08,2.5,Escola Privada,Ampla Concorrência,NaN,Não,Não,Sim,"Sim, recebendo entre meio e um salário mínimo",Não,Entre quatro e cinco salários mínimos,3,Sim,Não,NaN,Muito caro,Ruim,vejo uma abrangência limitada,Não,Sim,ansiedade


from matplotlib import pyplot as plt
dados1['idade'].plot(kind='hist', bins=20, title='idade')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
dados1['tempoNaUnifeiEmAnos'].plot(kind='hist', bins=20, title='tempoNaUnifeiEmAnos')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
dados1['contribuintes'].plot(kind='hist', bins=20, title='contribuintes')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(dados1['orientacaoSexual'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(dados1, x='idade', y='orientacaoSexual', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(dados1['etnia/cor'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(dados1, x='idade', y='etnia/cor', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(dados1['religiao'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(dados1, x='idade', y='religiao', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)